# Gradioの日！

今日は、とてつもなくシンプルなGradioフレームワークを使用してUIを構築します。喜びの準備をしてください！

注：Gradioの画面は、コンピューターの設定に応じて「ダークモード」または「ライトモード」で表示される場合があります。

この Jupyter Notebook（week2/day2.ipynb）の内容の概要は以下の通り。

主な流れは以下の通りです：

1. Gradioの導入
   - Gradioとは、Pythonで簡単にWebベースのUIを構築できるフレームワークです。
   - 最初は、テキストを大文字に変換するだけの関数`shout`をGradioでUI化する例から始まります。

2. APIキーの設定と主要ライブラリのインポート
   - OpenAI、Anthropic、Google Generative AIのAPIキーを環境変数から読み込み、接続確認。
   - 必要なライブラリ（requests, BeautifulSoup, dotenv, openai, gradioなど）をインポート。

3. 各種LLMへの問い合わせ関数
   - OpenAI GPT-4o-miniを使う`message_gpt`関数や、ストリーミングで返す`stream_gpt`関数。
   - Anthropic Claudeを使う`stream_claude`関数など。

4. GradioによるUIのバリエーション
   - ローカル実行、public（share=True）での公開、ダークモード強制などGradioの機能紹介。
   - 入力欄や出力欄のラベル付け、Markdownでの出力にも対応。

5. 複数モデル切り替え機能
   - モデル（GPTかClaude）をUI上で選択できるようにし、どちらのLLMでも使える仕組みに拡張。

6. Webページ解析と会社パンフレット自動生成
   - 指定した企業URLのHTMLを取得・解析し、タイトルや本文テキストを抽出。
   - その内容を元に、GPTまたはClaudeに「会社パンフレットを作成して」と依頼し、生成結果をMarkdownで表示。
   - GradioのUIから会社名・URL・モデルを選択してパンフレット生成を体験できる。

まとめ：
このノートブックは、Gradioの使い方と、複数LLMを組み合わせたインタラクティブなアプリ（例えば会社パンフレット自動生成）の作り方を、基礎から応用まで順を追って学べる内容になっています。PythonやGradioの初学者でも実践しやすい構成です。

In [1]:
# import

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
#import google.generativeai
#import anthropic

In [ ]:
# https://zenn.dev/bleed/articles/272c35ebfbac80
# GUI起動・・・・したが何も表示されない。どうやらバージョンが上がったことで、gradioが最新版になっているのが問題らしい。

#!pip install gradio
!echo y | pip uninstall gradio
!pip install gradio==5.33.1

In [2]:
import gradio as gr # そうそう！

In [3]:
# .envというファイルに環境変数をロードします
# キープレフィックスを印刷して、デバッグに役立ちます

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI APIキーが存在し、開始します {openai_api_key[:8]}")
else:
    print("Openai APIキーが設定されていません")
    
if anthropic_api_key:
    print(f"Anthropic APIキーが存在し、開始します {anthropic_api_key[:7]}")
else:
    print("人為的APIキーが設定されていません")

if google_api_key:
    print(f"Google APIキーが存在し、開始します {google_api_key[:8]}")
else:
    print("Google APIキーが設定されていません")

OpenAI APIキーが存在し、開始します sk-proj-
人為的APIキーが設定されていません
Google APIキーが設定されていません


In [4]:
# OpenAI, Anthropic と Googleに接続します。あなたがそれらを使用していない場合、Anthropic と Googleの行をコメントアウトしてください

openai = OpenAI()
#claude = anthropic.Anthropic()
#google.generativeai.configure()

In [5]:
# 一般的なシステムメッセージ - これ以上snarkyな敵対的なAIはありません！

system_message = "あなたは役に立つアシスタントです"

In [6]:
# 単純な機能でGPT-4o-miniへの呼び出しを包みましょう

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [7]:
# これにより、「トレーニングの遮断」、またはトレーニングデータの最新の日付が明らかになります

message_gpt("今日の日付は何ですか?")

'今日の日付は2023年10月4日です。'

## ユーザーインターフェイス時間！

In [8]:
# これが単純な機能です

def shout(text):
    print(f"入力によりシャウトが呼び出されました {text}") # ログ的な
    return text.upper()                                # shout = upper

In [9]:
shout("Hello")

入力によりシャウトが呼び出されました Hello


'HELLO'

In [10]:
# Gradioのシンプルな。これは「ライトモード」に表示される可能性があります - これを後でダークモードで作成する方法を示します。

gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [11]:
# share=Trueを追加すると、公開的にアクセスできる（ngrokのトンネル技術で、fnは引き続きローカルで動く）
# Huggingfaceのスペースと呼ばれるプラットフォームを使用して、より永続的なホスティングを利用できます。
# 注：一部のウイルス対策ソフトウェアと企業のファイアウォールは、share = trueを好まないかもしれません。
#    ワークネットワークで作業している場合は、このテストをスキップすることをお勧めします。

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://774a5392ffbafe795e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


入力によりシャウトが呼び出されました hello


In [12]:
# Inbrowser = trueを追加すると、新しいブラウザウィンドウが自動的に開きます

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


gio: http://127.0.0.1:7862/: Operation not supported


入力によりシャウトが呼び出されました hello
入力によりシャウトが呼び出されました hello


## ダークモードの強制
Gradio は、ブラウザとコンピュータの設定に応じて、ライトモードまたはダークモードで表示されます。Gradio を強制的にダークモードで表示する方法もありますが、これはユーザーの好み（特にアクセシビリティ上の理由）に委ねられるべきであるため、Gradio では推奨していません。画面をダークモードに強制的に切り替えたい場合は、以下の手順をお試しください。

In [13]:
# この変数を定義し、インターフェイスを作成するときにjs = force_dark_modeを渡す

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


入力によりシャウトが呼び出されました hello


In [14]:
# 入力と出力

view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


In [15]:
# そして今 - 関数を「叫び」から「message_gpt」に変更する

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


入力によりシャウトが呼び出されました hello


In [16]:
# マークダウンを使用しましょう
# その下のコードで言及されていない場合、System_messageを設定することがなぜ違いをもたらすのか疑問に思っていますか？
# message_gpt関数に戻って使用されているグローバル変数であるsystem_messageを利用しています（見てください）
# 優れたソフトウェアエンジニアリングの実践ではありませんが、Jupyter Lab R＆Dの間は非常に一般的です！

system_message = "あなたはマークダウンで応答する便利なアシスタントです"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")], # Textbox → Markdown
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [17]:
# 結果をストリーミングするコールを作成しましょう
# ジェネレーターの復習（「収量」キーワード）が必要な場合は、
# Week1フォルダーの中間Pythonノートブックをご覧ください。

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [18]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


In [19]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [20]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


## マイナーな改善

このコードを少し改善しました。

以前は、これらの行がありました。

```
for chunk in result:
  yield chunk
```

実際にこれを達成するためのよりエレガントな方法があります（Pythonの人々はもっと「Pythonic」と呼ぶかもしれません）：

「結果からの利回り」

これについては、Week1フォルダーの中間Pythonノートブックで詳細に説明します。詳細をご覧ください。

In [21]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    #elif model=="Claude":
    #    result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [22]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


# 会社のパンフレットジェネレーターの構築

今、あなたは方法を知っています - それは簡単です！

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">次のセルを読む前に、</h2>
            <span style="color:#900;">
                自分でも試してみましょう。第1週の5日目の会社パンフレットに戻って、最後にGradio UIを追加してみましょう。そして、ソリューションを見てみましょう。
            </span>
        </td>
    </tr>
</table>

In [23]:
# ウェブページを表すクラス

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [24]:
# ビル・Gに感謝します。ビル・Gは、これの以前のバージョンにバグがあることに気付きました！今修正されました。

system_message = "あなたは、企業のウェブサイトのランディングページのコンテンツを分析し、見込み顧客、投資家、\
採用候補者向けに企業に関する短いパンフレットを作成するアシスタントです。マークダウン形式で回答してください。"

In [25]:
def stream_brochure(company_name, url, model):
    yield ""
    prompt = f"{company_name} の会社パンフレットを作成してください。ランディングページはこちらです。\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    #elif model=="Claude":
    #    result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [26]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.
